In [32]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging

import torch
from torch import nn

from modules.until_module import PreTrainedModel, AllGather, CrossEn
from modules.module_cross import CrossModel, CrossConfig, Transformer as TransformerClip

from modules.module_clip import CLIP, convert_weights
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals
from __future__ import print_function

import os
from torch.utils.data import Dataset
import numpy as np
import pickle
from dataloaders.rawvideo_util import RawVideoExtractor
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from dataloaders.dataloader_msrvtt_retrieval import MSRVTT_DataLoader
from dataloaders.dataloader_msrvtt_retrieval import MSRVTT_TrainDataLoader
from dataloaders.dataloader_msvd_retrieval import MSVD_DataLoader
from dataloaders.dataloader_lsmdc_retrieval import LSMDC_DataLoader
from dataloaders.dataloader_activitynet_retrieval import ActivityNet_DataLoader
from dataloaders.dataloader_didemo_retrieval import DiDeMo_DataLoader
from modules.tokenization_clip import SimpleTokenizer as ClipTokenizer
from dataloaders.data_dataloaders import DATALOADER_DICT
import collections 
from collections import OrderedDict
from torch import nn
import import_ipynb 

from collections import OrderedDict
from typing import Tuple, Union

import hashlib
import os
import urllib
import warnings
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch import nn

In [34]:
_MODELS = {
    "RN50": "https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt",
    "RN101": "https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt",
    "RN50x4": "https://openaipublic.azureedge.net/clip/models/7e526bd135e493cef0776de27d5f42653e6b4c8bf9e0f653bb11773263205fdd/RN50x4.pt",
    "RN50x16": "https://openaipublic.azureedge.net/clip/models/52378b407f34354e150460fe41077663dd5b39c54cd0bfd2b27167a4a06ec9aa/RN50x16.pt",
    "ViT-B/32": "https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt",
    "ViT-B/16": "https://openaipublic.azureedge.net/clip/models/5806e77cd80f8b59890b7e101eabd078d9fb84e6937f9e85e4ecb61988df416f/ViT-B-16.pt",
}
_PT_NAME = {
    "RN50": "RN50.pt",
    "RN101": "RN101.pt",
    "RN50x4": "RN50x4.pt",
    "RN50x16": "RN50x16.pt",
    "ViT-B/32": "ViT-B-32.pt",
    "ViT-B/16": "ViT-B-16.pt",
}
def _download(url: str, root: str = os.path.expanduser("~/.cache/clip")):
    os.makedirs(root, exist_ok=True)
    filename = os.path.basename(url)

    expected_sha256 = url.split("/")[-2]
    download_target = os.path.join(root, filename)

    if os.path.exists(download_target) and not os.path.isfile(download_target):
        raise RuntimeError(f"{download_target} exists and is not a regular file")

    if os.path.isfile(download_target):
        if hashlib.sha256(open(download_target, "rb").read()).hexdigest() == expected_sha256:
            return download_target
        else:
            warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")

    with urllib.request.urlopen(url) as source, open(download_target, "wb") as output:
        with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True) as loop:
            while True:
                buffer = source.read(8192)
                if not buffer:
                    break

                output.write(buffer)
                loop.update(len(buffer))

    if hashlib.sha256(open(download_target, "rb").read()).hexdigest() != expected_sha256:
        raise RuntimeError(f"Model has been downloaded but the SHA256 checksum does not not match")

    return download_target

In [3]:
import easydict 
DATA_PATH = "/home/key2317/CLIP4Clip/msvd_data"
FEATURES_PATH = "/home/key2317/CLIP4Clip/msvd_data/MSVD_Videos"
args = easydict.EasyDict({
    "data_path":DATA_PATH,
    "features_path":FEATURES_PATH,
    "max_words":30,
    "feature_framerate":1,
    "max_frames":16,
    "image_resolution":224,
    "frame_order":0,
    "slice_framepos":0,
    "train_frame_order":0, #default 0, choice = [0,1,2]
    "batch_size":256,
    "n_gpu":torch.cuda.device_count(), #default :1
    "num_thread_reader":1,
    "datatype":"msvd",
    "eval_frame_order":0, #choices = [0, 1, 2]
    "batch_size_val":3500,
})

print(args.__dict__)
tokenizer = ClipTokenizer()
#train_dataloader, train_length, train_sampler = DATALOADER_DICT[args.datatype]["train"](args, tokenizer)

{'data_path': '/home/key2317/CLIP4Clip/msvd_data', 'features_path': '/home/key2317/CLIP4Clip/msvd_data/MSVD_Videos', 'max_words': 30, 'feature_framerate': 1, 'max_frames': 16, 'image_resolution': 224, 'frame_order': 0, 'slice_framepos': 0, 'train_frame_order': 0, 'batch_size': 256, 'n_gpu': 7, 'num_thread_reader': 1, 'datatype': 'msvd', 'eval_frame_order': 0, 'batch_size_val': 3500}


In [4]:
descriptions = ''
cross_model_name = 'cross-base'
n_gpu=1
cache_dir=""
pretrained_clip_name = "ViT-B/32"


# 회의 get_config는 module_clip 내에 선언되어 있는 CLIP 클래스에 정의되어 있음.
clip_state_dict = CLIP.get_config(pretrained_clip_name=pretrained_clip_name)
CONFIG_NAME = 'cross_config.json'
#print(clip_state_dict.keys())
action = 'store_true'
import easydict 
DATA_PATH = "/home/key2317/CLIP4Clip/msvd_data"
FEATURES_PATH = "/home/key2317/CLIP4Clip/msvd_data/MSVD_Videos"
args = easydict.EasyDict({
    "data_path":DATA_PATH,
    "features_path":FEATURES_PATH,
    "max_words":30,
    "feature_framerate":1,
    "max_frames":100,
    "image_resolution":224,
    "frame_order":0,
    "slice_framepos":0,
    "train_frame_order":0, #default 0, choice = [0,1,2]
    "batch_size":256,
    "n_gpu":torch.cuda.device_count(), #default :1
    "num_thread_reader":1,
    "datatype":"msvd",
    "eval_frame_order":0, #choices = [0, 1, 2]
    "batch_size_val":3500,
    "local_rank":0,
})

type_vocab_size = 2 
task_config = args
cross_config, _ = CrossConfig.get_config(cross_model_name, cache_dir, type_vocab_size, state_dict=None, task_config=task_config)
#print(args.__dict__)
tokenizer = ClipTokenizer()
#train_dataloader, train_length, train_sampler = DATALOADER_DICT[args.datatype]["train"](args, tokenizer)
print(args.__dict__)
tokenizer = ClipTokenizer()
#train_dataloader, train_length, train_sampler = DATALOADER_DICT[args.datatype]["train"](args, tokenizer)
cut_top_layer = 0

{'data_path': '/home/key2317/CLIP4Clip/msvd_data', 'features_path': '/home/key2317/CLIP4Clip/msvd_data/MSVD_Videos', 'max_words': 30, 'feature_framerate': 1, 'max_frames': 100, 'image_resolution': 224, 'frame_order': 0, 'slice_framepos': 0, 'train_frame_order': 0, 'batch_size': 256, 'n_gpu': 7, 'num_thread_reader': 1, 'datatype': 'msvd', 'eval_frame_order': 0, 'batch_size_val': 3500, 'local_rank': 0}


In [5]:
logger = logging.getLogger(__name__)
allgather = AllGather.apply
def print_shape(target):
    print("current shape {}".format(target.shape))

modeule_clip.py에 있는 내용 

Bottleneck : 연산량을 줄이기 위함

BottleNeck

In [35]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1):
        super().__init__()

        # all conv layers have stride 1. an avgpool is performed after the second convolution when stride > 1
        self.conv1 = nn.Conv2d(inplanes, planes, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.avgpool = nn.AvgPool2d(stride) if stride > 1 else nn.Identity()

        self.conv3 = nn.Conv2d(planes, planes * self.expansion, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = None
        self.stride = stride

        if stride > 1 or inplanes != planes * Bottleneck.expansion:
            # downsampling layer is prepended with an avgpool, and the subsequent convolution has stride 1
            self.downsample = nn.Sequential(OrderedDict([
                ("-1", nn.AvgPool2d(stride)),
                ("0", nn.Conv2d(inplanes, planes * self.expansion, 1, stride=1, bias=False)),
                ("1", nn.BatchNorm2d(planes * self.expansion))
            ]))

    def forward(self, x: torch.Tensor):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.avgpool(out)
        out = self.bn3(self.conv3(out))

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

#### ModifiedRsNet

In [33]:
class ModifiedResNet(nn.Module):
    """
    A ResNet class that is similar to torchvision's but contains the following changes:
    - There are now 3 "stem" convolutions as opposed to 1, with an average pool instead of a max pool.
    - Performs anti-aliasing strided convolutions, where an avgpool is prepended to convolutions with stride > 1
    - The final pooling layer is a QKV attention instead of an average pool
    """

    def __init__(self, layers, output_dim, heads, input_resolution=224, width=64):
        super().__init__()
        self.output_dim = output_dim
        self.input_resolution = input_resolution

        # the 3-layer stem
        self.conv1 = nn.Conv2d(3, width // 2, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width // 2)
        self.conv2 = nn.Conv2d(width // 2, width // 2, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(width // 2)
        self.conv3 = nn.Conv2d(width // 2, width, kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(width)
        self.avgpool = nn.AvgPool2d(2)
        self.relu = nn.ReLU(inplace=True)

        # residual layers
        self._inplanes = width  # this is a *mutable* variable used during construction
        self.layer1 = self._make_layer(width, layers[0])
        self.layer2 = self._make_layer(width * 2, layers[1], stride=2)
        self.layer3 = self._make_layer(width * 4, layers[2], stride=2)
        self.layer4 = self._make_layer(width * 8, layers[3], stride=2)

        embed_dim = width * 32  # the ResNet feature dimension
        self.attnpool = AttentionPool2d(input_resolution // 32, embed_dim, heads, output_dim)

    def _make_layer(self, planes, blocks, stride=1):
        layers = [Bottleneck(self._inplanes, planes, stride)]

        self._inplanes = planes * Bottleneck.expansion
        for _ in range(1, blocks):
            layers.append(Bottleneck(self._inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        def stem(x):
            for conv, bn in [(self.conv1, self.bn1), (self.conv2, self.bn2), (self.conv3, self.bn3)]:
                x = self.relu(bn(conv(x)))
            x = self.avgpool(x)
            return x

        x = x.type(self.conv1.weight.dtype)
        x = stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.attnpool(x)

        return x

#### CLIP4ClipPreTrainedModel

In [6]:
class CLIP4ClipPreTrainedModel(PreTrainedModel, nn.Module):
    """ An abstract class to handle weights initialization and
        a simple interface for dowloading and loading pretrained models.
    """
    def __init__(self, cross_config, *inputs, **kwargs):
        super(CLIP4ClipPreTrainedModel, self).__init__(cross_config)
        self.cross_config = cross_config
        self.clip = None
        self.cross = None

    @classmethod
    def from_pretrained(cls, cross_model_name, state_dict=None, cache_dir=None, type_vocab_size=2, *inputs, **kwargs):

        task_config = None
        if "task_config" in kwargs.keys():
            task_config = kwargs["task_config"]
            ###### 20220524 local rank
            if not hasattr(task_config, "local_rank"):
                #args.__dict__["max_frames"]출력하면 100 나옴을 확인
                task_config.__dict__["local_rank"] = 0
            elif task_config.local_rank == -1:
                task_config.local_rank = 0

        if state_dict is None: state_dict = {}
        pretrained_clip_name = "ViT-B/32"
        if hasattr(task_config, 'pretrained_clip_name'):
            pretrained_clip_name = task_config.pretrained_clip_name
        clip_state_dict = CLIP.get_config(pretrained_clip_name=pretrained_clip_name)
        for key, val in clip_state_dict.items():
            new_key = "clip." + key
            if new_key not in state_dict:
                state_dict[new_key] = val.clone()

        cross_config, _ = CrossConfig.get_config(cross_model_name, cache_dir, type_vocab_size, state_dict=None, task_config=task_config)

        model = cls(cross_config, clip_state_dict, *inputs, **kwargs)

        ## ===> Initialization trick [HARD CODE]
        if model.linear_patch == "3d":
            contain_conv2 = False
            for key in state_dict.keys():
                if key.find("visual.conv2.weight") > -1:
                    contain_conv2 = True
                    break
            if contain_conv2 is False and hasattr(model.clip.visual, "conv2"):
                cp_weight = state_dict["clip.visual.conv1.weight"].clone()
                kernel_size = model.clip.visual.conv2.weight.size(2)
                conv2_size = model.clip.visual.conv2.weight.size()
                conv2_size = list(conv2_size)

                left_conv2_size = conv2_size.copy()
                right_conv2_size = conv2_size.copy()
                left_conv2_size[2] = (kernel_size - 1) // 2
                right_conv2_size[2] = kernel_size - 1 - left_conv2_size[2]

                left_zeros, right_zeros = None, None
                if left_conv2_size[2] > 0:
                    left_zeros = torch.zeros(*tuple(left_conv2_size), dtype=cp_weight.dtype, device=cp_weight.device)
                if right_conv2_size[2] > 0:
                    right_zeros = torch.zeros(*tuple(right_conv2_size), dtype=cp_weight.dtype, device=cp_weight.device)

                cat_list = []
                if left_zeros != None: cat_list.append(left_zeros)
                cat_list.append(cp_weight.unsqueeze(2))
                if right_zeros != None: cat_list.append(right_zeros)
                cp_weight = torch.cat(cat_list, dim=2)

                state_dict["clip.visual.conv2.weight"] = cp_weight

        if model.sim_header == 'tightTransf':
            contain_cross = False
            for key in state_dict.keys():
                if key.find("cross.transformer") > -1:
                    contain_cross = True
                    break
            if contain_cross is False:
                for key, val in clip_state_dict.items():
                    if key == "positional_embedding":
                        state_dict["cross.embeddings.position_embeddings.weight"] = val.clone()
                        continue
                    if key.find("transformer.resblocks") == 0:
                        num_layer = int(key.split(".")[2])

                        # cut from beginning
                        if num_layer < task_config.cross_num_hidden_layers:
                            state_dict["cross."+key] = val.clone()
                            continue

        if model.sim_header == "seqLSTM" or model.sim_header == "seqTransf":
            contain_frame_position = False
            for key in state_dict.keys():
                if key.find("frame_position_embeddings") > -1:
                    contain_frame_position = True
                    break
            if contain_frame_position is False:
                for key, val in clip_state_dict.items():
                    if key == "positional_embedding":
                        state_dict["frame_position_embeddings.weight"] = val.clone()
                        continue
                    if model.sim_header == "seqTransf" and key.find("transformer.resblocks") == 0:
                        num_layer = int(key.split(".")[2])
                        # cut from beginning
                        if num_layer < task_config.cross_num_hidden_layers:
                            state_dict[key.replace("transformer.", "transformerClip.")] = val.clone()
                            continue
        ## <=== End of initialization trick

        if state_dict is not None:
            model = cls.init_preweight(model, state_dict, task_config=task_config)

        return model

Task_config와 Target_config

In [7]:
def show_log(task_config, info):
    if task_config is None or task_config.local_rank == 0:
        logger.warning(info)

def update_attr(target_name, target_config, target_attr_name, source_config, source_attr_name, default_value=None):
    if hasattr(source_config, source_attr_name):
        if default_value is None or getattr(source_config, source_attr_name) != default_value:
            setattr(target_config, target_attr_name, getattr(source_config, source_attr_name))
            show_log(source_config, "Set {}.{}: {}.".format(target_name,
                                                            target_attr_name, getattr(target_config, target_attr_name)))
    return target_config

def check_attr(target_name, task_config):
    return hasattr(task_config, target_name) and task_config.__dict__[target_name]

DATALoader 생성하여 video의 shape(6차원)을 확인 
video shape : 
 - Pair : 1
 - max_frame : 100 
 - batch : 1 
 - Channel : 3 
 - H : 224
 - W : 224

In [8]:
def dataloader_msvd_train(args,tokenizer):
    msvd_dataset=MSVD_DataLoader(
        subset = "train",
        data_path = args.data_path,
        features_path = args.features_path,
        max_words = args.max_words,
        feature_framerate=args.feature_framerate,
        tokenizer = tokenizer,
        max_frames=args.max_frames,
        frame_order = args.train_frame_order, 
        slice_framepos = args.slice_framepos
    )

    #train_sampler = torch.utils.data.distributed.DistributedSampler(msvd_dataset)
    train_sampler = 0
    dataloader = DataLoader(
        msvd_dataset,
        batch_size = args.batch_size // args.n_gpu, 
        num_workers = args.num_thread_reader,
        pin_memory=False, 
        shuffle = (train_sampler is None), 
        sampler = train_sampler, 
        drop_last=True,

        
    )

    return msvd_dataset, dataloader, len(msvd_dataset),train_sampler

def dataloader_msvd_test(args, tokenizer, subset="test"):
    msvd_testset = MSVD_DataLoader(
        subset=subset,
        data_path=args.data_path,
        features_path=args.features_path,
        max_words=args.max_words,
        feature_framerate=args.feature_framerate,
        tokenizer=tokenizer,
        max_frames=args.max_frames,
        frame_order=args.eval_frame_order,
        slice_framepos=args.slice_framepos,
    )
    dataloader_msvd = DataLoader(
        msvd_testset,
        batch_size=args.batch_size_val,
        num_workers=args.num_thread_reader,
        shuffle=False,
        drop_last=False,
    )
    return msvd_testset, dataloader_msvd, len(msvd_testset)

In [9]:
msvd_dataset,train_dataloader,len_of_msvdtrain,train_sampler = dataloader_msvd_train(args,tokenizer)
msvd_testset, test_dataloader, len_of_msvdtest = dataloader_msvd_test(args,tokenizer,subset="test")

Video number: 1200
Total Paire: 48774
For test, sentence number: 27763
For test, video number: 670
Video number: 670
Total Paire: 27763


DATA_PATH와  FEATURE_PATH를 확인

In [10]:
print("DATA PATH :",DATA_PATH)
print("FEATURE PATH ",FEATURES_PATH)
video_id_path_dict = {}
video_id_path_dict["train"] = os.path.join(DATA_PATH, "train_list.txt")
video_id_path_dict["val"] = os.path.join(DATA_PATH, "val_list.txt")
video_id_path_dict["test"] = os.path.join(DATA_PATH, "test_list.txt")
caption_file = os.path.join(DATA_PATH, "raw-captions.pkl")

DATA PATH : /home/key2317/CLIP4Clip/msvd_data
FEATURE PATH  /home/key2317/CLIP4Clip/msvd_data/MSVD_Videos


captions : video_id와 대사로 이루어진 dict

video_ids : video_id를 모아놓은 리스트

In [11]:
with open(caption_file,'rb') as f:
    captions = pickle.load(f)

with open(video_id_path_dict["train"], 'r') as fp:
    video_ids = [itm.strip() for itm in fp.readlines()]

video_dict : 파일과 video_id로 이루어진 dict 



In [12]:
video_dict = {} 
for root, dub_dir,video_files in os.walk(args.features_path):
    for video_file in video_files:
        video_id_ = ".".join(video_file.split(".")[:-1])
        if video_id_ not in video_ids:
            continue
        file_path_ = os.path.join(root,video_file)
        video_dict[video_id_] = file_path_ 

#print(video_dict)

sentences_dict : key : 인덱스 / value : (video_id, 문장) 으로 이루어진 dict

In [13]:
sentences_dict = {} 
cut_off_points=[] 
for video_id in video_ids:
    assert video_id in captions 
    for cap in captions[video_id]:
        cap_txt = " ".join(cap)
        sentences_dict[len(sentences_dict)] = (video_id,cap_txt)
    cut_off_points.append(len(sentences_dict))
#print(sentences_dict)
#print(cut_off_points)
print(len(sentences_dict))

48774


msvd_dataset에서 getitem을 할 때, 
 - get_text의 shape
 - get_video의 shape

를 확인하는 작업입니다.

dataset에서 _getitem_을 통해 인덱스 하나에 대하여 video와 text를 각각 꺼내옵니다.

1. get_text

In [14]:
idx = 1 
video_id,caption = sentences_dict[idx]
print(video_id)
print(caption)
pairs_text, pairs_mask, pairs_segment, choice_video_ids  = msvd_dataset._get_text(video_id,caption)

print(choice_video_ids)



-4wsuPCjDBc_5_15
a chipmunk is eating
['-4wsuPCjDBc_5_15']


2. get_video

In [15]:
video, video_mask = msvd_dataset._get_rawvideo(choice_video_ids)
print(video.shape)
print(video_mask.shape)

raw_video_data_clip shape : torch.Size([11, 3, 224, 224])
raw_video_slice shape : torch.Size([11, 1, 3, 224, 224])
(1, 100, 1, 3, 224, 224)
(1, 100)


3. getitem

In [16]:
pairs_text,pairs_mask,pairs_segment,video,video_mask = msvd_dataset.__getitem__(1)


raw_video_data_clip shape : torch.Size([11, 3, 224, 224])
raw_video_slice shape : torch.Size([11, 1, 3, 224, 224])


In [17]:
class AttentionPool2d(nn.Module):

    def __init__(self, spacial_dim: int, embed_dim : int, num_heads : int, output_dim : int = None):
        super(AttentionPool2d,self).__init__()
        #spacial_dim의 제곱 + 1 , embed_dim 
        self.positional_embedding = nn.Parameter(torch.randn(spacial_dim**2 +1 ,embed_dim) / embed_dim ** 0.5)
        print(self.positional_embedding.shape)
        # Key, Query, Value
        self.k_proj = nn.Linear(embed_dim,embed_dim)
        self.q_proj = nn.Linear(embed_dim,embed_dim)
        self.v_proj = nn.Linear(embed_dim,embed_dim)
        
        self.c_proj = nn.Linear(embed_dim,output_dim or embed_dim)
        self.num_heads = num_heads 

    def forward(self,x):
        ###################### 차원 한번 줄임 ######################
        x = x.reshape(x.shape[0],x.shape[1],x.shape[2] * x.shape[3]).permute(2,0,1) # NCHW -> (HW)NC
        print('permuted x shape : {}'.format(x.shape))
        x = torch.cat([x.mean(dim=0,keepdim=True),x],dim=0) #(HW+1)NC
        print('torch cat x shape :',x.shape) #(224 * 224 +1 , 1, 3)
        x = x + self.positional_embedding[:,None, :].to(x.dtype) # (HW+1)NC
        print("added with positional_embedding shape :",x.shape)
        ## multi head attention 수행
        x, _ = F.multi_head_attention_forward(
            query = x, key = x, value = x, 
            
            embed_dim_to_check = x.shape[-1],
            num_heads = self.num_heads, 

            q_proj_weight = self.q_proj.weight, #nn.Linear(embed_dim,embed_dim) 의 weight
            k_proj_weight = self.k_proj.weight, #Linear 
            v_proj_weight = self.v_proj.weight, 
            in_proj_weight = None, 
            in_proj_bias = torch.cat([self.q_proj.bias, self.k_proj.bias, self.v_proj.bias]), 
            bias_k = None,
            bias_v = None, 
            add_zero_attn = False, 
            dropout_p= 0, 
            out_proj_weight=self.c_proj.weight, 
            out_proj_bias= self.c_proj.bias, 
            use_separate_proj_weight=True, 
            training = self.training, 
            need_weights = False
        )

        return x[0]


embed_dim = clip_state_dict["text_projection"].shape[1]

context_length = clip_state_dict["positional_embedding"].shape[0]

vocab_size = clip_state_dict["token_embedding.weight"].shape[0]

transformer_width = clip_state_dict["ln_final.weight"].shape[0]

transformer_heads = transformer_width // 64

#transformer_layers = len(set(k.split(".")[2] for k in clip_state_dict if k.startswith(f"transformer.resblocks"))) #12

transformer_layers = 12


AttentionPool2d의 spec 
 - spacial_dim : 224 
 - embed_dim : 3 
 - num_heads : 3 
 - output_dim : 512 
 - embedding 의 shape : (spacial_dim) * (spacial_dim) , 1 , 3 

 - input x의 shape : (N : 1, C : 3, H : 224, W : 224)
 - output x[0]의 shape : 1 , output_dim 512

Vision Transformer 

In [18]:
class LayerNorm(nn.LayerNorm):

    def forward(self,x : torch.Tensor):
        orig_type = x.dtype 
        ret = super().forward(x.type(torch.float32))
        return ret.type(orig_type)

class QuickGELU(nn.Module):
    def forward(self,x:torch.Tensor):
        return x * torch.sigmoid(1.702 * x)

ReisudalAttentionBlock : 크게 다를 것은 없음.

In [19]:
class ResidualAttentionBlock(nn.Module):
    # d_model이 Transformer에서 넘어온 width와 같음.
    # # d_model == width  
    def __init__(self, d_model : int, n_head : int, attn_mask = None):
        super(ResidualAttentionBlock,self).__init__()
        
        ##### attention Block
        self.attn = nn.MultiheadAttention(d_model,n_head)
        self.ln_1 = LayerNorm(d_model)
        self.mlp = nn.Sequential(OrderedDict([
            ("c_fc",nn.Linear(d_model,d_model*4)),
            ("gelu",QuickGELU()),
            ("c_proj",nn.Linear(d_model * 4, d_model))
        ]))
        self.ln_2 = LayerNorm(d_model)
        self.attn_mask = attn_mask 
    
    def attention(self,x:torch.Tensor):
        attn_mask_ = self.attn_mask 
        if self.attn_mask is not None and hasattr(self.attn_mask, '__call__'):
            attn_mask_ = self.attn_mask(x.size(0))
        attn_mask_ = attn_mask_.to(dtype=x.dtype,device = x.device) if attn_mask_ is not None else None 
        return self.attn(x,x,x,need_weights=False, attn_mask = attn_mask_)[0]
    
    def forward(self,x_tuple : tuple):
        x,video_frame = x_tuple  
        #print("Residualattention video_frame :",video_frame)
        x = x + self.attention(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return (x,video_frame)

In [20]:
class Transformer(nn.Module):
    def __init__(self, width :int, layers :int , heads :int, attn_mask = None):
        super(Transformer,self).__init__()
        self.width = width 
        self.layers = layers 
        # layer의 숫자만큼 Residualattention을 진행
        self.resblocks = nn.Sequential(*[ResidualAttentionBlock(width,heads,attn_mask) for _ in range(layers)])

    def forward(self, x: torch.Tensor, video_frame = -1):
        return self.resblocks((x,video_frame))[0]

#### AutoEncoder

#### VisionTransformer : 

transformer_width : clip_state_dict["ln_final.weight"].shape[0]

transformer_heads : transformer_width// 64 64등분!

In [21]:
class VisualTransformer(nn.Module):
    def __init__(self,input_resolution : int, patch_size : int, width : int, layers : int, heads : int, output_dim : int, linear_patch : str = '2d',):
        super(VisualTransformer,self).__init__()
        self.input_resolution = input_resolution 
        self.output_dim = output_dim 

        ##### 2D일 때에는 Conv1d
        self.conv1 = nn.Conv2d(in_channels=3, out_channels = width, kernel_size = patch_size,stride=patch_size,bias=False)

        scale = width ** -0.5
        self.class_embedding = nn.Parameter(scale * torch.randn(width))
        self.positional_embedding = nn.Parameter(scale * torch.randn((input_resolution // patch_size)**2 +1, width)) 
        self.ln_pre = LayerNorm(width)


        #Transformer 인자 1 : width / 인자 2 : layers / 인자 3 : heads 
        self.transformer = Transformer(width,layers,heads)


        self.ln_post = LayerNorm(width)
        self.proj = nn.Parameter(scale * torch.randn(width,output_dim))   

        assert linear_patch in ['2d','3d']
        self.linear_patch = linear_patch 
        if self.linear_patch == '3d':
            #### 3D일 때에는 Conv2d
            self.conv2 = nn.Conv3d(in_channels=3, out_channels=width, kernel_size = (3,patch_size,patch_size), 
            stride = (1,patch_size,patch_size),padding = (1,0,0),bias= False)
    # Task 2 !!! 
    def forward(self, x: torch.Tensor, video_frame=-1):
            ####Conv3D AE 20220526 #####
            #x_3d shape : 
            x_3d = x.reshape(-1, video_frame, x.shape[-3], x.shape[-2], x.shape[-1])
            print("1111 x_3d shape :",x_3d.shape)
            if self.linear_patch == '3d':
                assert video_frame != -1
                x_3d = x.reshape(-1, video_frame, x.shape[-3], x.shape[-2], x.shape[-1])
                print("2222 x_3d shape :",x_3d.shape)
                x_3d = x_3d.permute(0, 2, 1, 3, 4) #
                print("3333 x_3d shape :",x_3d.shape)
                #x_3d = x_3d.permute()
                x_3d = self.conv2(x_3d)     # shape = [*, width, frame, grid, grid]
                print("4444 x_3d shape :",x_3d.shape)
                x_3d = x_3d.permute(0, 2, 1, 3, 4)      # shape = [*, frame, width, grid, grid]
                print("5555 x_3d shape :",x_3d.shape)
                x = x_3d.reshape(-1, x_3d.shape[-3], x_3d.shape[-2], x_3d.shape[-1]).contiguous() # shape = [*, width, grid, grid]
                print("666 x_3d shape :",x_3d.shape)
            else:
                x = self.conv1(x)  # shape = [*, width, grid, grid]

            x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]
            x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]

            #################################### 20220607 여기까지는 구현 성공함 ##################################
            x = torch.cat([self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # shape = [*, grid ** 2 + 1, width]
            x = x + self.positional_embedding.to(x.dtype) 
            x = self.ln_pre(x)
            print("<<<< x shape : >>>>>",x.shape)
            ################# TAE!!! #################### 
            x = x.permute(1, 0, 2)  # NLD -> LND
            ############### 원래 기존으로 가 #######################
            x = self.transformer(x, video_frame=video_frame)
            x = x.permute(1, 0, 2)  # LND -> NLD

            # Move the three lines below to `encode_image` for entire hidden sequence
            # x = self.ln_post(x[:, 0, :])
            # if self.proj is not None:
            #     x = x @ self.proj

            return x



CLIP 해부

1. CLIP 초기화에 필요한 Vision parameter

In [22]:

vision_layers = len([k for k in clip_state_dict.keys() if k.startswith("visual.") and k.endswith(".attn.in_proj_weight")])
vision_width = clip_state_dict["visual.conv1.weight"].shape[0]
vision_patch_size = clip_state_dict["visual.conv1.weight"].shape[-1]
grid_size = round((clip_state_dict["visual.positional_embedding"].shape[0] - 1) ** 0.5)
image_resolution = vision_patch_size * grid_size # 32 x 7 
vision_heads = vision_width //64
linear_patch = '2d'

2. CLIP 초기화에 필요한 text parameter

In [23]:
embed_dim = clip_state_dict["text_projection"].shape[1]
context_length = clip_state_dict["positional_embedding"].shape[0]
vocab_size = clip_state_dict["token_embedding.weight"].shape[0]
transformer_width = clip_state_dict["ln_final.weight"].shape[0]
transformer_heads = transformer_width // 64
#transformer_layers = len(set(k.split(".")[2] for k in clip_state_dict if k.startswith(f"transformer.resblocks"))) #12
transformer_layers = 12


In [24]:
token_embedding = nn.Embedding(vocab_size,transformer_width)
positional_embedding = nn.Parameter(torch.empty(context_length,transformer_width))
ln_final = LayerNorm(transformer_width)
text_projection = nn.Parameter(torch.empty(transformer_width,embed_dim))
logit_scale = nn.Parameter(torch.ones([]))


3. CLIP 내부에 있는 모든 ViT, text transformer 객체 만들기

In [25]:
########### visual transformer
visual = VisualTransformer(
    input_resolution = image_resolution, 
    patch_size=vision_patch_size,
    width=vision_width,
    layers=vision_layers,
    heads=vision_heads,
    output_dim=embed_dim,
    linear_patch=linear_patch    
)




def build_attention_mask(context_length):
# lazily create causal attention mask, with full attention between the vision tokens
# pytorch uses additive attention mask; fill with -inf
    mask = torch.zeros(context_length, context_length)
    mask.fill_(float("-inf"))
    mask.triu_(1)  # zero out the lower diagonal
    return mask


############ Text transformer
transformer = Transformer(
    width=transformer_width,
    layers=transformer_layers,
    heads=transformer_heads,
    attn_mask=build_attention_mask
)

4. Text와 image를 Encode하는 함수, 그리고 이를 받아서 forward하는 함수

In [26]:
@property
def dtype():
    return visual.conv1.weight.dtype

def encode_image(image, return_hidden=False, video_frame=-1):
    hidden = visual(image.type(dtype), video_frame=video_frame)
    hidden = visual.ln_post(hidden) @ visual.proj

    x = hidden[:, 0, :]

    if return_hidden:
        return x, hidden

    return x

def encode_text(text, return_hidden=False):
    x = token_embedding(text).type(dtype)  # [batch_size, n_ctx, d_model]

    pos_emd = positional_embedding[:x.size(1), :].type(dtype)
    x = x + pos_emd
    x = x.permute(1, 0, 2)  # NLD -> LND
    x = transformer(x)
    x = x.permute(1, 0, 2)  # LND -> NLD

    hidden = ln_final(x).type(dtype) @ text_projection

    # x.shape = [batch_size, n_ctx, transformer.width]
    # take features from the eot embedding (eot_token is the highest number in each sequence)
    x = hidden[torch.arange(hidden.shape[0]), text.argmax(dim=-1)]

    if return_hidden:
        return x, hidden

    return x

def forward(image, text):
    # cae = CAE() # encoder : 차원 축소, autoencoder : encoder + decoder
    # encoded, decoded = encoder(image)

    # image_features = encoded_image(encoded)
    # text_features = encoded_text(text)

    # reconstruction_error = mse_loss(encoded, decoded)
    # return logits_per_image, logits_per_text, reconstruction_error
    # (t2v + v2tv)/2 + reconstruction_error, or (total loss) / 3
    
    ####################### 20220531 CAE #####################
    image_features = encode_image(image)
    text_features = encode_text(text)

    # normalized features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # cosine similarity as logits
    logit_scale = logit_scale.exp()
    logits_per_image = logit_scale * image_features @ text_features.t()
    logits_per_text = logit_scale * text_features @ image_features.t()

    # shape = [global_batch_size, global_batch_size]
    return logits_per_image, logits_per_text

#### AutoEncoder 3D

In [27]:
class CAE(nn.Module):
    # T x C x H X W
    def __init__(self, frames, ch, width, height):
        super(CAE, self).__init__()
        self.frames = frames
        self.ch = ch
        self.width = width
        self.height = height
        def EncoderLayer(in_channel, out_channel, pooling_size=None):
            network = []
            #network += [TImeDistributed(nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1)]
            
            network += [nn.Conv3d(in_channel, out_channel, kernel_size=3, padding=1)]
            network += [nn.BatchNorm3d(out_channel)]
            network += [nn.LeakyReLU()]
            if pooling_size is not None:
                network += [nn.MaxPool3d(kernel_size=pooling_size)]
            network = nn.Sequential(*network)
            return network
        def DecoderLayer(in_channel, out_channel, size=None):
            network = []
            if size is None:
                network += [nn.Conv3d(in_channel, out_channel, kernel_size=3, padding=1)]
            else:
                network += [nn.ConvTranspose3d(in_channel, out_channel, kernel_size=size, padding=0, stride=size)]
            network += [nn.BatchNorm3d(out_channel)]
            network += [nn.LeakyReLU()]
            network = nn.Sequential(*network)
            return network
        self.encoder1 = EncoderLayer(3, 32, pooling_size=2)
        self.encoder2 = EncoderLayer(32, 48, pooling_size=2)
        self.encoder3 = EncoderLayer(48, 64)
        ##################################
        

        ##################################
        self.decoder1 = DecoderLayer(64, 48)
        self.decoder2 = DecoderLayer(48, 32, size=2)
        self.decoder3 = DecoderLayer(32, 3, size=2)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        #encoded = self.encoder3(self.encoder2(self.encoder1(x)))
        x = self.encoder1(x)
        #print("encoder1 통과했을 때 x shape : ",x.shape)
        x = self.encoder2(x)
        #print("encoder2 통과했을 때 x shape :",x.shape)
        encoded = self.encoder3(x)
        #torch.Size([1, 64, 56, 56, 4])
        res_encoded = encoded.permute(0,4,2,3,1) # 1, 4, 56, 56, 64
        res_encoded = res_encoded.permute(0,1,4,3,2)
        #print('마지막 encoded shape :',encoded.shape)
        decoded = self.decoder3(self.decoder2(self.decoder1(encoded)))
        
        decoded = self.sigmoid(decoded)
        #print("decodered x shape ",decoded.shape)
        return res_encoded, decoded
    def compute_loss(self, x, y):
        mse = nn.MSELoss()
        loss = mse(x, y)
        return loss, loss.item()



# encoded shape : torch.Size([10, 64, 56, 56, 3])
# decoded shape : torch.Size([10, 3, 224, 224, 12])
# image shape toward VIT -> ch : 64, h : 56, w : 56, f : 3 
# (not same as oirignal shape (3, 224, 224, 12) -> have to solove the different shape
# otherwise, (B, C, H, W ,F) = (B, 3, 224, 224, 1) (if not solved, jh tries this method)

In [28]:
set = CAE(frames=16,ch=3,width=image_resolution,height=image_resolution)
video = torch.randn(1,3,224,224,16)
#video = torch.randn(1,16,3,224,224)
res_encoded, decoded = set.forward(video)



CLIP4Clip에 정의된 get_visual_output

In [36]:
class CLIP(nn.Module):
    def __init__(self,
                 embed_dim: int,
                 # vision
                 input_resolution: int,
                 vision_layers: Union[Tuple[int, int, int, int], int],
                 vision_width: int,
                 vision_patch_size: int,
                 # text
                 context_length: int,
                 vocab_size: int,
                 transformer_width: int,
                 transformer_heads: int,
                 transformer_layers: int,
                 # vision linear of patch
                 linear_patch: str = '2d',
                 ):
        super().__init__()

        self.context_length = context_length

        if isinstance(vision_layers, (tuple, list)):
            vision_heads = vision_width * 32 // 64
            self.visual = ModifiedResNet(
                layers=vision_layers,
                output_dim=embed_dim,
                heads=vision_heads,
                input_resolution=input_resolution,
                width=vision_width
            )
        else:
            vision_heads = vision_width // 64
            self.visual = VisualTransformer(
                input_resolution=input_resolution,
                patch_size=vision_patch_size,
                width=vision_width,
                layers=vision_layers,
                heads=vision_heads,
                output_dim=embed_dim,
                linear_patch=linear_patch
            )

        self.transformer = Transformer(
            width=transformer_width,
            layers=transformer_layers,
            heads=transformer_heads,
            attn_mask=self.build_attention_mask
        )

        self.vocab_size = vocab_size
        self.token_embedding = nn.Embedding(vocab_size, transformer_width)
        self.positional_embedding = nn.Parameter(torch.empty(self.context_length, transformer_width))
        self.ln_final = LayerNorm(transformer_width)

        self.text_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))
        self.logit_scale = nn.Parameter(torch.ones([]))

        self.initialize_parameters()

    def initialize_parameters(self):
        nn.init.normal_(self.token_embedding.weight, std=0.02)
        nn.init.normal_(self.positional_embedding, std=0.01)

        if isinstance(self.visual, ModifiedResNet):
            if self.visual.attnpool is not None:
                std = self.visual.attnpool.c_proj.in_features ** -0.5
                nn.init.normal_(self.visual.attnpool.q_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.k_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.v_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.c_proj.weight, std=std)

            for resnet_block in [self.visual.layer1, self.visual.layer2, self.visual.layer3, self.visual.layer4]:
                for name, param in resnet_block.named_parameters():
                    if name.endswith("bn3.weight"):
                        nn.init.zeros_(param)

        proj_std = (self.transformer.width ** -0.5) * ((2 * self.transformer.layers) ** -0.5)
        attn_std = self.transformer.width ** -0.5
        fc_std = (2 * self.transformer.width) ** -0.5
        for block in self.transformer.resblocks:
            nn.init.normal_(block.attn.in_proj_weight, std=attn_std)
            nn.init.normal_(block.attn.out_proj.weight, std=proj_std)
            nn.init.normal_(block.mlp.c_fc.weight, std=fc_std)
            nn.init.normal_(block.mlp.c_proj.weight, std=proj_std)

        if self.text_projection is not None:
            nn.init.normal_(self.text_projection, std=self.transformer.width ** -0.5)

    @staticmethod
    def get_config(pretrained_clip_name="ViT-B/32"):
        model_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), "ViT-B-32.pt")
        if pretrained_clip_name in _MODELS and pretrained_clip_name in _PT_NAME:
            model_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), _PT_NAME[pretrained_clip_name])

        if pretrained_clip_name in ["ViT-B/32", "ViT-B/16"] and os.path.exists(model_path):
            pass
        else:
            if pretrained_clip_name in _MODELS:
                model_path = _download(_MODELS[pretrained_clip_name])
            elif os.path.isfile(pretrained_clip_name):
                model_path = pretrained_clip_name
            else:
                raise RuntimeError(f"Model {pretrained_clip_name} not found; available models = {available_models()}")

        try:
            # loading JIT archive
            model = torch.jit.load(model_path, map_location="cpu").eval()
            state_dict = model.state_dict()
        except RuntimeError:
            state_dict = torch.load(model_path, map_location="cpu")

        return state_dict

    def build_attention_mask(self, context_length):
        # lazily create causal attention mask, with full attention between the vision tokens
        # pytorch uses additive attention mask; fill with -inf
        mask = torch.zeros(context_length, context_length)
        mask.fill_(float("-inf"))
        mask.triu_(1)  # zero out the lower diagonal
        return mask

    @property
    def dtype(self):
        return self.visual.conv1.weight.dtype
    
    # input : video(4D) video_frame : bs * ts
    def encode_input(self, input, return_hidden=False, video_frame=-1):

        hidden = self.visual(input.type(self.dtype), video_frame=video_frame)
        hidden = self.visual.ln_post(hidden) @ self.visual.proj

        x = hidden[:, 0, :]

        if return_hidden:
            return x, hidden

        return x

    def encode_text(self, text, return_hidden=False):
        x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]

        pos_emd = self.positional_embedding[:x.size(1), :].type(self.dtype)
        x = x + pos_emd
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD

        hidden = self.ln_final(x).type(self.dtype) @ self.text_projection

        # x.shape = [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = hidden[torch.arange(hidden.shape[0]), text.argmax(dim=-1)]

        if return_hidden:
            return x, hidden

        return x

    def forward(self, input, text):
        input_features = self.encode_input(input)
        text_features = self.encode_text(text)

        # normalized features
        input_features = input_features / input_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        # cosine similarity as logits
        logit_scale = self.logit_scale.exp()
        logits_per_input = logit_scale * input_features @ text_features.t()
        logits_per_text = logit_scale * text_features @ input_features.t()

        # shape = [global_batch_size, global_batch_size]
        return logits_per_input, logits_per_text

In [ ]:
clip = CLIP(
            embed_dim,
            image_resolution, vision_layers-cut_top_layer, vision_width, vision_patch_size,
            context_length, vocab_size, transformer_width, transformer_heads, transformer_layers-cut_top_layer,
            linear_patch=linear_patch
        ).float()



In [ ]:
vit = VisualTransformer(
input_resolution = image_resolution, 
patch_size = vision_patch_size, 
width = vision_width, 
layers = vision_layers,
heads= vision_heads,
output_dim=embed_dim, 
linear_patch=linear_patch)

In [ ]:
print("image resolution :",image_resolution)
print('vision_patch_size :',vision_patch_size)
print('vision_width :',vision_width)
print("vision_layers :",vision_layers)
print("vision_heads :",vision_heads)
print("vision_heads :",vision_heads)
print("output_dim :",embed_dim)
print("linear patch :",linear_patch)

image resolution : 224
vision_patch_size : 32
vision_width : 768
vision_layers : 12
vision_heads : 12
vision_heads : 12
output_dim : 512
linear patch : 2d


#### VisionAETransformer

In [30]:
class AUTOENCODER(nn.Module):
    def __init__(self, channel, width : int, height : int, linear_patch='3d'):
        super(AUTOENCODER,self).__init__()
        self.width  = width 
        self.height = height 
        assert linear_patch in ['2d','3d']
        self.linear_patch = linear_patch

        def EncoderLayer(in_channel,out_channel,pooling_size=None):
            network=[] 

            network += [nn.Conv3d(in_channel, out_channel, kernel_size=3, padding=1)]
            network += [nn.BatchNorm3d(out_channel)]
            network += [nn.LeakyReLU()]
            if pooling_size is not None:
                network += [nn.MaxPool3d(kernel_size=pooling_size)]
            network = nn.Sequential(*network)
            return network

        def DecoderLayer(in_channel,out_channel,size=None):
            network = [] 
            if size is None:
                network +=[nn.Conv3d(in_channel,out_channel,kernel_size=3,padding=1)]
            else:
                network +=[nn.ConvTranspose3d(in_channel,out_channel,kernel_size=size,padding=0,stride=size)]
            network +=[nn.BatchNorm3d(out_channel)]
            network += [nn.LeakyReLU()]
            network = nn.Sequential(*network)
            return network
        
        self.encoder1 = EncoderLayer(channel, 32, pooling_size=2)
        self.encoder2 = EncoderLayer(32, 48, pooling_size=2)
        self.encoder3 = EncoderLayer(48, 64)
        self.decoder1 = DecoderLayer(64, 48)
        self.decoder2 = DecoderLayer(48, 32, size=2)
        self.decoder3 = DecoderLayer(32, channel, size=2)
        self.sigmoid = nn.Sigmoid()

        # case 20220607 : self.encoder = EncoderLayer(48, 768) & not used conv2
    
    def forward(self, x):
        x = self.encoder1(x)
        x = self.encoder2(x)
        encoded = self.encoder3(x)
        decoded = self.decoder3(self.decoder2(self.decoder1(encoded))) 
        decoded = self.sigmoid(decoded)
        return encoded, decoded
        
    def compute_loss(self, x, y):
        mse = nn.MSELoss()
        loss = mse(x, y)
        return loss, loss.item()

class VAeT(nn.Module):
    def __init__(self,input_resolution : int, patch_size : int, width : int, layers : int, heads : int, output_dim : int, linear_patch :str = '2d'):
        super(VAeT, self).__init__()

        self.AE = AUTOENCODER(channel=3,width=image_resolution,height=image_resolution)

        self.input_resolution = input_resolution
        self.output_dim = output_dim 

        ### 2d일 때에는 Conv1d 
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=width,kernel_size = patch_size,stride=patch_size, bias=False)

        scale = width ** -0.5 
        self.class_embedding = nn.Parameter(scale * torch.randn(width))
        self.positional_embedding = nn.Parameter(scale * torch.randn(65, width))
        self.ln_pre = LayerNorm(width)
        
        
        self.transformer = Transformer(width,layers,heads)

        self.ln_post = LayerNorm(width)
        self.proj = nn.Parameter(scale *torch.randn(width,output_dim))

        assert linear_patch in ['2d','3d']
        self.linear_patch = linear_patch 

        if self.linear_patch == '3d':
            self.conv2 = nn.Conv3d(in_channels=64,out_channels=vision_width,kernel_size=(7,7,1),
                stride = (7,7,1),padding=(0,1,0),bias=False)

    def forward(self, x:torch.Tensor,video_frame=-1):
        video_frame = x.shape[1] # video frame
        
        encoded, decoded = self.AE(x)

        # for VIT
        x = self.conv2(encoded)
        x = x.permute(0,4,1,3,2)
        x = x.reshape(-1,x.shape[-3],x.shape[-2]*x.shape[-1]).contiguous()
        x = x.permute(0,2,1)
        x = torch.cat([self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # shape = [*, grid ** 2 + 1, width]
        x = x + self.positional_embedding.to(x.dtype) 
        x = self.ln_pre(x)
        x = x.permute(1,0,2)
        x = self.transformer(x,video_frame=3)
        x = x.permute(1,0,2)

        return x, decoded

In [31]:
model = VAeT(input_resolution=image_resolution,patch_size=vision_patch_size,width=vision_width,
layers = vision_layers,heads=vision_heads,output_dim=embed_dim,linear_patch='3d')

sample_video = torch.randn(1,3,224,224,12) # NLD x channel x width x height x frame 
# res_encoded,decoded = test.forward(sample_video)

res_encoded, decoded = model(sample_video)
print("encoded and VAeT res shape :",res_encoded.shape)

encoded and VAeT res shape : torch.Size([3, 65, 768])
